# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f7d98ed75e0>
├── 'a' --> tensor([[-0.5268, -1.3699,  0.6357],
│                   [ 0.9015,  0.4279,  0.9879]])
└── 'x' --> <FastTreeValue 0x7f7d98ed74f0>
    └── 'c' --> tensor([[-0.8814, -1.1687,  0.2131,  0.5581],
                        [-0.9686, -0.4543, -0.8380, -1.2562],
                        [ 0.0797, -0.3318, -1.3079, -0.8710]])

In [4]:
t.a

tensor([[-0.5268, -1.3699,  0.6357],
        [ 0.9015,  0.4279,  0.9879]])

In [5]:
%timeit t.a

75.7 ns ± 0.0283 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f7d98ed75e0>
├── 'a' --> tensor([[-0.6373, -0.4900, -0.2376],
│                   [-0.4853,  0.9092, -1.3758]])
└── 'x' --> <FastTreeValue 0x7f7d98ed74f0>
    └── 'c' --> tensor([[-0.8814, -1.1687,  0.2131,  0.5581],
                        [-0.9686, -0.4543, -0.8380, -1.2562],
                        [ 0.0797, -0.3318, -1.3079, -0.8710]])

In [7]:
%timeit t.a = new_value

77.6 ns ± 0.216 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.5268, -1.3699,  0.6357],
               [ 0.9015,  0.4279,  0.9879]]),
    x: Batch(
           c: tensor([[-0.8814, -1.1687,  0.2131,  0.5581],
                      [-0.9686, -0.4543, -0.8380, -1.2562],
                      [ 0.0797, -0.3318, -1.3079, -0.8710]]),
       ),
)

In [10]:
b.a

tensor([[-0.5268, -1.3699,  0.6357],
        [ 0.9015,  0.4279,  0.9879]])

In [11]:
%timeit b.a

59.7 ns ± 0.0447 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-1.1073, -0.1042,  0.9332],
               [ 0.4432, -0.0116,  0.8417]]),
    x: Batch(
           c: tensor([[-0.8814, -1.1687,  0.2131,  0.5581],
                      [-0.9686, -0.4543, -0.8380, -1.2562],
                      [ 0.0797, -0.3318, -1.3079, -0.8710]]),
       ),
)

In [13]:
%timeit b.a = new_value

494 ns ± 0.0736 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

806 ns ± 0.127 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

10.7 µs ± 26.6 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

143 µs ± 201 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

144 µs ± 715 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f7cee336bb0>
├── 'a' --> tensor([[[-0.5268, -1.3699,  0.6357],
│                    [ 0.9015,  0.4279,  0.9879]],
│           
│                   [[-0.5268, -1.3699,  0.6357],
│                    [ 0.9015,  0.4279,  0.9879]],
│           
│                   [[-0.5268, -1.3699,  0.6357],
│                    [ 0.9015,  0.4279,  0.9879]],
│           
│                   [[-0.5268, -1.3699,  0.6357],
│                    [ 0.9015,  0.4279,  0.9879]],
│           
│                   [[-0.5268, -1.3699,  0.6357],
│                    [ 0.9015,  0.4279,  0.9879]],
│           
│                   [[-0.5268, -1.3699,  0.6357],
│                    [ 0.9015,  0.4279,  0.9879]],
│           
│                   [[-0.5268, -1.3699,  0.6357],
│                    [ 0.9015,  0.4279,  0.9879]],
│           
│                   [[-0.5268, -1.3699,  0.6357],
│                    [ 0.9015,  0.4279,  0.9879]]])
└── 'x' --> <FastTreeValue 0x7f7d98ea5130>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.1 µs ± 52.6 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f7cedbc9700>
├── 'a' --> tensor([[-0.5268, -1.3699,  0.6357],
│                   [ 0.9015,  0.4279,  0.9879],
│                   [-0.5268, -1.3699,  0.6357],
│                   [ 0.9015,  0.4279,  0.9879],
│                   [-0.5268, -1.3699,  0.6357],
│                   [ 0.9015,  0.4279,  0.9879],
│                   [-0.5268, -1.3699,  0.6357],
│                   [ 0.9015,  0.4279,  0.9879],
│                   [-0.5268, -1.3699,  0.6357],
│                   [ 0.9015,  0.4279,  0.9879],
│                   [-0.5268, -1.3699,  0.6357],
│                   [ 0.9015,  0.4279,  0.9879],
│                   [-0.5268, -1.3699,  0.6357],
│                   [ 0.9015,  0.4279,  0.9879],
│                   [-0.5268, -1.3699,  0.6357],
│                   [ 0.9015,  0.4279,  0.9879]])
└── 'x' --> <FastTreeValue 0x7f7cedc301f0>
    └── 'c' --> tensor([[-0.8814, -1.1687,  0.2131,  0.5581],
                        [-0.9686, -0.4543, -0.8380, -1.2562],
                 

In [23]:
%timeit t_cat(trees)

30.1 µs ± 35.3 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

56.7 µs ± 50 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-0.5268, -1.3699,  0.6357],
                [ 0.9015,  0.4279,  0.9879]],
       
               [[-0.5268, -1.3699,  0.6357],
                [ 0.9015,  0.4279,  0.9879]],
       
               [[-0.5268, -1.3699,  0.6357],
                [ 0.9015,  0.4279,  0.9879]],
       
               [[-0.5268, -1.3699,  0.6357],
                [ 0.9015,  0.4279,  0.9879]],
       
               [[-0.5268, -1.3699,  0.6357],
                [ 0.9015,  0.4279,  0.9879]],
       
               [[-0.5268, -1.3699,  0.6357],
                [ 0.9015,  0.4279,  0.9879]],
       
               [[-0.5268, -1.3699,  0.6357],
                [ 0.9015,  0.4279,  0.9879]],
       
               [[-0.5268, -1.3699,  0.6357],
                [ 0.9015,  0.4279,  0.9879]]]),
    x: Batch(
           c: tensor([[[-0.8814, -1.1687,  0.2131,  0.5581],
                       [-0.9686, -0.4543, -0.8380, -1.2562],
                       [ 0.0797, -0.3318, -1.3079, -0.8710]],
         

In [26]:
%timeit Batch.stack(batches)

78.3 µs ± 211 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-0.5268, -1.3699,  0.6357],
               [ 0.9015,  0.4279,  0.9879],
               [-0.5268, -1.3699,  0.6357],
               [ 0.9015,  0.4279,  0.9879],
               [-0.5268, -1.3699,  0.6357],
               [ 0.9015,  0.4279,  0.9879],
               [-0.5268, -1.3699,  0.6357],
               [ 0.9015,  0.4279,  0.9879],
               [-0.5268, -1.3699,  0.6357],
               [ 0.9015,  0.4279,  0.9879],
               [-0.5268, -1.3699,  0.6357],
               [ 0.9015,  0.4279,  0.9879],
               [-0.5268, -1.3699,  0.6357],
               [ 0.9015,  0.4279,  0.9879],
               [-0.5268, -1.3699,  0.6357],
               [ 0.9015,  0.4279,  0.9879]]),
    x: Batch(
           c: tensor([[-0.8814, -1.1687,  0.2131,  0.5581],
                      [-0.9686, -0.4543, -0.8380, -1.2562],
                      [ 0.0797, -0.3318, -1.3079, -0.8710],
                      [-0.8814, -1.1687,  0.2131,  0.5581],
                      [-0.9686, -

In [28]:
%timeit Batch.cat(batches)

143 µs ± 343 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

327 µs ± 964 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
